In [1]:
from Crypto.Util.number import *
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad, pad

p = 14668080038311483271
C_list = [[11315841881544731102, 2283439871732792326, 6800685968958241983, 6426158106328779372, 9681186993951502212],
          [4729583429936371197, 9934441408437898498, 12454838789798706101, 1137624354220162514, 8961427323294527914],
          [12212265161975165517, 8264257544674837561, 10531819068765930248, 4088354401871232602, 14653951889442072670],
          [6045978019175462652, 11202714988272207073, 13562937263226951112, 6648446245634067896, 13902820281072641413],
          [1046075193917103481, 3617988773170202613, 3590111338369894405, 2646640112163975771, 5966864698750134707]]
D_list = [[1785348659555163021, 3612773974290420260, 8587341808081935796, 4393730037042586815, 10490463205723658044],
          [10457678631610076741, 1645527195687648140, 13013316081830726847, 12925223531522879912, 5478687620744215372],
          [9878636900393157276, 13274969755872629366, 3231582918568068174, 7045188483430589163, 5126509884591016427],
          [4914941908205759200, 7480989013464904670, 5860406622199128154, 8016615177615097542, 13266674393818320551],
          [3005316032591310201, 6624508725257625760, 7972954954270186094, 5331046349070112118, 6127026494304272395]]
msg = b"\xcc]B:\xe8\xbc\x91\xe2\x93\xaa\x88\x17\xc4\xe5\x97\x87@\x0fd\xb5p\x81\x1e\x98,Z\xe1n`\xaf\xe0%:\xb7\x8aD\x03\xd2Wu5\xcd\xc4#m'\xa7\xa4\x80\x0b\xf7\xda8\x1b\x82k#\xc1gP\xbd/\xb5j"

Fp = GF(p)
C_mat = matrix(Fp, C_list)
D_mat = matrix(Fp, D_list)

#计算 C 和 D 的特征值
eval_C = C_mat.eigenvalues()
eval_D = D_mat.eigenvalues()

#去掉0特征值
eval_C_nonzero = [x for x in eval_C if x != 0]
eval_D_nonzero = [x for x in eval_D if x != 0]

#输出特征值
print("Nonzero eigenvalues of C:", eval_C_nonzero)
print("Nonzero eigenvalues of D:", eval_D_nonzero)

#尝试计算离散对数以找到key
key = None
for mu in eval_D_nonzero:
    try:
        #尝试计算第一个非零特征值对应的k值
        k = discrete_log(mu, eval_C_nonzero[0])
        #检查k是否满足条件
        valid = True
        for lam in eval_C_nonzero:
            if lam^k not in eval_D_nonzero:
                valid = False
                break
        if valid:
            key = k
            print(f"Found key: {key}")
            break
    except Exception as e:
        print(f"Error: {e}")
        continue

if key is None:
    raise ValueError("Key not found")

key_bytes = pad(long_to_bytes(key), 16)
aes = AES.new(key_bytes, AES.MODE_ECB)
flag = unpad(aes.decrypt(msg), 64)
print("Flag:", flag)

Nonzero eigenvalues of C: [13548047239731931439, 10741008122066331899, 2915915082365181132, 2524362820657834710]
Nonzero eigenvalues of D: [14219969811373602463, 7805278355513795080, 7126986745593039829, 6321945571561295171]
Found key: 5273966641785501202
Flag: b'LILCTF{Are_y0u_5till_4wake_que5t1on_m4ker!}'
